In [ ]:
# YouTube 影片字幕生成器 (Gemini 版本)

## 1. 安裝必要套件


In [ ]:
# 安裝必要的套件
!pip install google-cloud-speech google-generativeai yt-dlp srt fastapi "uvicorn[standard]" nest-asyncio pyngrok


In [ ]:
# 設定 ngrok
from pyngrok import ngrok
# ngrok.set_auth_token('你的_authtoken')  # 取消註解並填入您的 token


In [ ]:
# 設定 ngrok
from pyngrok import ngrok
# ngrok.set_auth_token('你的_authtoken')  # 取消註解並填入您的 token


In [ ]:
# 設定 ngrok
from pyngrok import ngrok
# ngrok.set_auth_token('你的_authtoken')  # 取消註解並填入您的 token


In [ ]:
# 設定 ngrok
from pyngrok import ngrok
# ngrok.set_auth_token('你的_authtoken')  # 取消註解並填入您的 token


In [ ]:
## 2. 設定 Google API Key


In [ ]:
import google.generativeai as genai

# 設定您的 Google API Key
GOOGLE_API_KEY = "your_api_key_here"  # 替換成您的 API Key
genai.configure(api_key=GOOGLE_API_KEY)

# 初始化 Gemini Pro 模型
model = genai.GenerativeModel('gemini-pro')


In [ ]:
## 3. 設定 ngrok（用於公開訪問）


In [ ]:
# 設定 ngrok
from pyngrok import ngrok
# ngrok.set_auth_token('你的_authtoken')  # 取消註解並填入您的 token


In [ ]:
## 4. 主程式


In [ ]:
from fastapi import FastAPI, Form, HTTPException
from fastapi.responses import StreamingResponse, HTMLResponse
from fastapi.middleware.cors import CORSMiddleware
from google.cloud import speech
import yt_dlp
import srt
import os
import tempfile
import datetime
import subprocess
from pathlib import Path
from typing import AsyncGenerator
import uvicorn
import nest_asyncio

# 初始化 FastAPI
app = FastAPI()

# 允許所有來源的跨域請求
app.add_middleware(
    CORSMiddleware,
    allow_origins=["*"],
    allow_credentials=True,
    allow_methods=["*"],
    allow_headers=["*"],
)

# 取得音訊長度（秒）
def get_audio_duration(audio_path: str) -> float:
    result = subprocess.run([
        "ffprobe",
        "-v", "error",
        "-show_entries", "format=duration",
        "-of", "default=noprint_wrappers=1:nokey=1",
        audio_path
    ], stdout=subprocess.PIPE)
    return float(result.stdout.decode().strip())

# 分割音訊
def split_audio(audio_path, segment_duration=15):
    output_dir = Path(audio_path).parent / "segments"
    output_dir.mkdir(exist_ok=True)

    total_duration = get_audio_duration(audio_path)
    segments = []

    for start_time in range(0, int(total_duration), segment_duration):
        segment_path = output_dir / f"segment_{start_time}.wav"  # 使用 WAV 格式
        cmd = [
            "ffmpeg",
            "-i", audio_path,
            "-ss", str(start_time),
            "-t", str(segment_duration),
            "-acodec", "pcm_s16le",  # 16-bit PCM
            "-ar", "16000",  # 16kHz 採樣率
            "-ac", "1",  # 單聲道
            str(segment_path),
            "-y"
        ]
        subprocess.run(cmd, check=True, capture_output=True)
        segments.append((start_time, str(segment_path)))

    return segments

# 使用 Google Speech-to-Text 進行轉錄
def transcribe_audio(audio_path: str) -> list:
    client = speech.SpeechClient()

    with open(audio_path, "rb") as audio_file:
        content = audio_file.read()

    audio = speech.RecognitionAudio(content=content)
    config = speech.RecognitionConfig(
        encoding=speech.RecognitionConfig.AudioEncoding.LINEAR16,
        sample_rate_hertz=16000,
        language_code="zh-TW",
        enable_automatic_punctuation=True,
    )

    response = client.recognize(config=config, audio=audio)
    
    results = []
    for result in response.results:
        for word_info in result.alternatives[0].words:
            results.append({
                "word": word_info.word,
                "start_time": word_info.start_time.total_seconds(),
                "end_time": word_info.end_time.total_seconds()
            })
    
    return results

# 使用 Gemini 優化字幕
def optimize_subtitles(text: str) -> str:
    prompt = f"""
    請優化以下字幕文本，使其更加通順和易讀，但保持原意不變：
    {text}
    """
    response = model.generate_content(prompt)
    return response.text

# 處理音訊片段
async def process_audio_segment_stream(segment_path: str, start_time: float) -> AsyncGenerator[str, None]:
    try:
        print(f"開始處理音訊片段：{segment_path}")
        
        # 使用 Google Speech-to-Text 轉錄
        words = transcribe_audio(segment_path)
        
        # 組合成句子
        current_sentence = []
        sentence_start = None
        
        for word in words:
            current_sentence.append(word["word"])
            if sentence_start is None:
                sentence_start = word["start_time"]
            
            # 根據標點符號或時間間隔判斷句子結束
            if any(p in word["word"] for p in "。！？.!?") or \
               (len(current_sentence) > 0 and word["end_time"] - sentence_start > 5):
                
                sentence_text = "".join(current_sentence)
                # 使用 Gemini 優化字幕
                optimized_text = optimize_subtitles(sentence_text)
                
                subtitle = srt.Subtitle(
                    index=1,
                    start=datetime.timedelta(seconds=sentence_start + start_time),
                    end=datetime.timedelta(seconds=word["end_time"] + start_time),
                    content=optimized_text
                )
                yield srt.compose([subtitle]) + "\n"
                
                current_sentence = []
                sentence_start = None

    except Exception as e:
        print(f"處理音訊片段時發生錯誤: {e}")
        raise

# API：產生字幕串流
@app.post("/api/generate-subtitle")
async def generate_subtitle(
    youtube_url: str = Form(None),
    segment_duration: int = Form(15)
):
    if not youtube_url:
        raise HTTPException(status_code=400, detail="請提供 YouTube 連結")

    async def generate():
        try:
            with tempfile.TemporaryDirectory() as tmpdir:
                # 下載 YouTube 音訊
                ydl_opts = {
                    'format': 'bestaudio/best',
                    'outtmpl': os.path.join(tmpdir, 'audio.%(ext)s'),
                    'postprocessors': [{
                        'key': 'FFmpegExtractAudio',
                        'preferredcodec': 'wav',
                        'preferredquality': '192',
                    }]
                }
                with yt_dlp.YoutubeDL(ydl_opts) as ydl:
                    ydl.download([youtube_url])
                
                audio_path = None
                for f in os.listdir(tmpdir):
                    if f.endswith(".wav"):
                        audio_path = os.path.join(tmpdir, f)
                        break

                if not audio_path:
                    raise HTTPException(status_code=500, detail="音訊檔案處理失敗")

                # 分段 + 逐段轉錄
                segments = split_audio(audio_path, segment_duration)
                for start_time, segment_path in segments:
                    async for subtitle in process_audio_segment_stream(segment_path, start_time):
                        yield subtitle

        except Exception as e:
            raise HTTPException(status_code=500, detail=f"字幕產生失敗: {str(e)}")

    return StreamingResponse(generate(), media_type="text/plain")

# 前端頁面（與之前相同的 HTML 內容）
@app.get("/")
async def read_root():
    html_content = """
    <!DOCTYPE html>
    <html lang="zh-TW">
    <head>
        <meta charset="UTF-8">
        <meta name="viewport" content="width=device-width, initial-scale=1.0">
        <title>YouTube 影片字幕生成器 (Gemini)</title>
        <style>
            body {
                font-family: 'Microsoft JhengHei', Arial, sans-serif;
                max-width: 1200px;
                margin: 0 auto;
                padding: 20px;
                background-color: #f5f5f5;
            }
            .container {
                background-color: white;
                padding: 30px;
                border-radius: 10px;
                box-shadow: 0 2px 10px rgba(0, 0, 0, 0.1);
            }
            h1 {
                color: #333;
                text-align: center;
                margin-bottom: 30px;
            }
            .input-section {
                display: flex;
                gap: 10px;
                margin-bottom: 30px;
            }
            #youtubeUrl {
                flex: 1;
                padding: 10px;
                border: 2px solid #ddd;
                border-radius: 5px;
                font-size: 16px;
            }
            #startBtn {
                padding: 10px 20px;
                background-color: #4285f4;
                color: white;
                border: none;
                border-radius: 5px;
                cursor: pointer;
                font-size: 16px;
                transition: background-color 0.3s;
            }
            #startBtn:hover {
                background-color: #3367d6;
            }
            .video-container {
                position: relative;
                width: 100%;
                max-width: 800px;
                margin: 0 auto;
            }
            #player {
                width: 100%;
                aspect-ratio: 16/9;
                margin-bottom: 20px;
            }
            #subtitle {
                background-color: rgba(0, 0, 0, 0.7);
                color: white;
                padding: 15px;
                text-align: center;
                font-size: 18px;
                min-height: 50px;
                border-radius: 5px;
                margin-top: 20px;
            }
        </style>
    </head>
    <body>
        <div class="container">
            <h1>YouTube 影片字幕生成器 (Gemini)</h1>
            
            <div class="input-section">
                <input type="text" id="youtubeUrl" placeholder="請輸入 YouTube 影片連結">
                <button id="startBtn">開始轉錄</button>
            </div>

            <div class="video-container">
                <div id="player"></div>
                <div id="subtitle">等待字幕生成...</div>
            </div>
        </div>

        <script src="https://www.youtube.com/iframe_api"></script>
        <script>
            let player;
            const subtitleDiv = document.getElementById("subtitle");
            let subtitles = [];
            let playerReady = false;
            let lastSubtitle = null;

            function onYouTubeIframeAPIReady() {
                player = new YT.Player("player", {
                    height: "360",
                    width: "640",
                    playerVars: {
                        'playsinline': 1,
                        'origin': window.location.origin,
                        'enablejsapi': 1,
                        'autoplay': 0,
                        'host': 'https://www.youtube.com'
                    },
                    events: {
                        onReady: (event) => {
                            playerReady = true;
                        },
                        onStateChange: onPlayerStateChange,
                        onError: (event) => {
                            console.error("Player error:", event.data);
                        }
                    }
                });
            }

            function onPlayerStateChange(event) {
                if (event.data == YT.PlayerState.PLAYING) {
                    updateSubtitle();
                }
            }

            function getYouTubeID(url) {
                const regExp = /^.*(?:youtu.be\/|v\/|u\/\w\/|embed\/|watch\?v=|&v=)([^#&?]*).*/;
                const match = url.match(regExp);
                return match && match[1].length === 11 ? match[1] : null;
            }

            function loadYouTubeVideo(videoId) {
                if (!playerReady) {
                    setTimeout(() => loadYouTubeVideo(videoId), 100);
                    return;
                }
                try {
                    player.loadVideoById(videoId);
                    player.playVideo();
                } catch (err) {
                    console.error("載入影片時發生錯誤：", err);
                    subtitleDiv.textContent = "載入影片時發生錯誤，請重新整理頁面後再試";
                }
            }

            function updateSubtitle() {
                if (!playerReady || !player || subtitles.length === 0) {
                    requestAnimationFrame(updateSubtitle);
                    return;
                }
                try {
                    const currentTime = player.getCurrentTime();
                    let currentSub = null;
                    for (let i = subtitles.length - 1; i >= 0; i--) {
                        if (currentTime >= subtitles[i].start && currentTime <= subtitles[i].end) {
                            currentSub = currentSub || subtitles[i];
                            break;
                        }
                    }
                    if (currentSub && (!lastSubtitle || lastSubtitle.text !== currentSub.text)) {
                        lastSubtitle = currentSub;
                        subtitleDiv.textContent = currentSub.text;
                    } else if (!currentSub && lastSubtitle) {
                        subtitleDiv.textContent = "";
                        lastSubtitle = null;
                    }
                    requestAnimationFrame(updateSubtitle);
                } catch (err) {
                    console.error("更新字幕時發生錯誤：", err);
                    requestAnimationFrame(updateSubtitle);
                }
            }

            document.getElementById("startBtn").onclick = async () => {
                const url = document.getElementById("youtubeUrl").value.trim();
                const videoId = getYouTubeID(url);

                if (!videoId) {
                    alert("請輸入正確的 YouTube 連結");
                    return;
                }

                subtitleDiv.textContent = "準備中...";
                subtitles = [];
                lastSubtitle = null;

                try {
                    const response = await fetch("/api/generate-subtitle", {
                        method: "POST",
                        headers: { "Content-Type": "application/x-www-form-urlencoded" },
                        body: new URLSearchParams({
                            youtube_url: url,
                            segment_duration: "15"
                        })
                    });

                    if (!response.ok) {
                        throw new Error(`HTTP error! status: ${response.status}`);
                    }

                    if (!response.body) {
                        subtitleDiv.textContent = "無法取得字幕串流";
                        return;
                    }

                    const reader = response.body.getReader();
                    const decoder = new TextDecoder("utf-8");
                    let buffer = "";
                    let subtitleCount = 0;
                    const startTime = Date.now();

                    // 開始字幕更新循環
                    updateSubtitle();

                    while (true) {
                        const { done, value } = await reader.read();
                        if (done) break;

                        buffer += decoder.decode(value, { stream: true });
                        const parts = buffer.split("\n\n");
                        buffer = parts.pop();

                        for (const part of parts) {
                            const lines = part.trim().split("\n");
                            if (lines.length < 3) continue;

                            const timeMatch = lines[1].match(/(\d{2}:\d{2}:\d{2},\d{3}) --> (\d{2}:\d{2}:\d{2},\d{3})/);
                            if (!timeMatch) continue;

                            function toSeconds(t) {
                                const [h, m, s, ms] = t.split(/[:,]/);
                                return (+h) * 3600 + (+m) * 60 + (+s) + (+ms) / 1000;
                            }

                            subtitles.push({
                                start: toSeconds(timeMatch[1]),
                                end: toSeconds(timeMatch[2]),
                                text: lines.slice(2).join("\n").trim()
                            });

                            subtitleCount++;
                            if (subtitleCount === 1) {
                                console.log("收到第一個字幕，字幕系統準備就緒");
                                subtitleDiv.textContent = "字幕系統準備就緒，開始播放影片";
                                loadYouTubeVideo(videoId);
                            }

                            if (subtitleCount % 10 === 0) {
                                const elapsedTime = ((Date.now() - startTime) / 1000).toFixed(1);
                                console.log(`已處理 ${subtitleCount} 個字幕 (${elapsedTime}秒)`);
                            }
                        }
                    }

                    console.log(`字幕生成完成。共 ${subtitles.length} 個字幕`);

                } catch (err) {
                    console.error("產生字幕時發生錯誤：", err);
                    subtitleDiv.textContent = "產生字幕時發生錯誤：" + err.message;
                }
            };
        </script>
    </body>
    </html>
    """
    return HTMLResponse(content=html_content)

# 設定 ngrok
ngrok_tunnel = ngrok.connect(8000)
print('公開網址:', ngrok_tunnel.public_url)

# 啟動伺服器
nest_asyncio.apply()
uvicorn.run(app, port=8000)
